# RAKE

Below is the implementation of RAKE Algorithm which extracts keywords from documents and remove stopwords also.

https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents

Above is the link to its paper

Reason to implement this
1. It reduces unnecessary tokens from the text which adds no meaning to text
2. Reduces the length of text which is beneficial for BERT now
3. Similar to implementation in IR bert paper

In [ ]:
pip install rake-nltk

In [ ]:
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
from rake_nltk import Rake,Metric


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/IR\ group2/

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1RUkqjxZuKOpUEkLIkLfd5Y5fv5UlC-q5/IR group2


In [ ]:
import pickle
with open('final_data.pkl', 'rb') as fp:
  doc = pickle.load(fp)

In [ ]:
import pickle
with open('final_query.pkl', 'rb') as fp:
  query = pickle.load(fp)

In [ ]:
def extract_key(text, r):
	
	r.extract_keywords_from_text(text) 
	a = r.get_ranked_phrases()
	t = ""
	for phrase in a:
		t = t + ' ' + phrase
	return t

In [ ]:
def extract_key(text, w, r):
	
	r.extract_keywords_from_text(text) 
	a = r.get_ranked_phrases()
	t = ""
	cut = min(w, len(a))
	count = 0
	for phrase in a:
		if count >= cut:
			break
		t = t + ' ' + phrase
		count += 1
	return t

In [ ]:
query_corpus = []
r = Rake()
for i in range(50): 
  key_text = extract_key(query[i] ,200, r)
  query_corpus.append(key_text)
  print(i)

In [ ]:
text_corpus = []
r = Rake()
for i in range(2914): 
  key_text = extract_key(doc[i] ,200, r)
  text_corpus.append(key_text)
  print(i)

In [ ]:
import pickle
with open('rake_text_corpus.pkl', 'wb') as fp:
  pickle.dump(text_corpus,fp)

# SBERT

Implementing SBERT with the help of predtrained embeddings from document text and query text and now computing the similarity score 

In [ ]:
pip install -U sentence-transformers

In [ ]:
import pickle
with open('passage_embedding_corpus.pkl', 'rb') as fp:
  passage_embedding_corpus= pickle.load(fp)

In [ ]:
import pickle
with open('q_emb.pkl', 'rb') as fp:
  query= pickle.load(fp)

In [ ]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilroberta-base-msmarco-v2')

In [ ]:
query=[]
for i in range(len(query_corpus)):
  query_embedding = model.encode(query_corpus[i])
  query.append(query_embedding)

In [ ]:
similarityarray=np.zeros(shape=(2914,50))
similaritydoc=0
for k in range(len(query)):
  for i in range(len(passage_embedding_corpus)):
    for j in range(len(passage_embedding_corpus[i])):
      similarity=util.pytorch_cos_sim(query[k], passage_embedding_corpus[i][j])
      similaritydoc=similaritydoc+similarity
    similaritydoc=similaritydoc/len(passage_embedding_corpus[i])
    similarityarray[i][k]=np.array(similaritydoc)
    similaritydoc=0

In [ ]:
print(similarityarray)

[[0.32289436 0.51817822 0.39361042 ... 0.49360007 0.51800758 0.51900643]
 [0.32116768 0.42965642 0.3708612  ... 0.47301197 0.43151057 0.41884822]
 [0.38772932 0.49233943 0.41018295 ... 0.52661622 0.495877   0.56453204]
 ...
 [0.26464292 0.23892173 0.23499994 ... 0.33462954 0.24709587 0.24972513]
 [0.41093251 0.43496338 0.3963114  ... 0.43778819 0.44921988 0.51846546]
 [0.27381629 0.43672079 0.38361862 ... 0.41312018 0.45438763 0.39208347]]


In [ ]:
import pickle
with open('sbert_cossim_rakequery.pkl', 'wb') as fp:
  pickle.dump(similarityarray,fp)

In [ ]:
similarityarray=np.zeros(shape=(2914,50))
similaritydoc=0
for k in range(len(query)):
  for i in range(len(passage_embedding_corpus)):
    for j in range(len(passage_embedding_corpus[i])):
      similarity=util.pytorch_cos_sim(query[k], passage_embedding_corpus[i][j])
      similaritydoc=similaritydoc+similarity
    similaritydoc=similaritydoc/len(passage_embedding_corpus[i])
    similarityarray[i][k]=np.array(similaritydoc)
    similaritydoc=0

In [ ]:
print(similarityarray)

[[0.32289442 0.51817811 0.39361039 ... 0.49359995 0.5180074  0.51900643]
 [0.32116771 0.42965639 0.3708612  ... 0.47301185 0.43151054 0.41884819]
 [0.38772935 0.49233934 0.41018298 ... 0.52661616 0.49587691 0.56453204]
 ...
 [0.26464292 0.23892169 0.23499998 ... 0.33462942 0.24709576 0.24972507]
 [0.41093263 0.43496329 0.39631143 ... 0.43778816 0.4492197  0.51846546]
 [0.27381632 0.43672076 0.38361856 ... 0.41312015 0.45438758 0.39208353]]
